<a href="https://colab.research.google.com/github/soflat11/Guidelines/blob/main/demo20240504.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 事前準備
このファイルは  
**5/12 ARDS診療ガイドライン2026_成人CQ**  
の解析に対応しています。  
この解析は、修正デルファイでの、同意の１～９点の集計に対応しており、他の集計、たとえばPatientsの文面の選択などの集計には対応していません。

以下の事前作業を投票前に済ませておいてください。

## Google Driveのマウント
Google driveをこのファイルで操作できるようにマウントさせます。
ショートカット内にフォルダがないと、生成したスライドがただしく保存できないので、あらかじめ、データが可能されている共有フォルダを自身のMyDrive内にショートカットを作成してください。

In [1]:
from google.colab import drive
drive.mount('/content/drive') #コラボでGoogle driveを操作できるようにしています

Mounted at /content/drive


## ライブラリの取得
解析に必要なライブラリをインポートします。
事前に、ライブラリがインストールされていない場合は＃を外して、ライブラリをインストールしてください。


In [ ]:

# python-pptx ライブラリのインストール（未インストールの場合）
!pip install python-pptx


# ライブラリのインポート
from pptx import Presentation
from pptx.util import Inches, Pt
import pandas as pd
import io
import matplotlib.pyplot as plt
import numpy as np
from pptx import Presentation
from pptx.util import Inches

## データの読込み
スプレッドシートを読み込みます。urlの読み込みでは、xlsxにフォーマとを変換して読み込んでいます。
スプレッドシートの共有ボタンを押すと、以下のURLがでます。

https://docs.google.com/spreadsheets/d/1MXIEA-pUss9VBQ1nVzmk83AKdXQwwOw1YbWLz_4LTMg/edit?usp=sharing

これは共有リンクです。
これの語尾をexport?format=xlsxにしてデータを読み込ませます。

pandasとopenpyxlライブラリを使ってスプレッドシートのデータを読み込むことができます。Colabノートブックに次のように記述します。

In [4]:
#データの読込
url = 'https://docs.google.com/spreadsheets/d/1MXIEA-pUss9VBQ1nVzmk83AKdXQwwOw1YbWLz_4LTMg/export?format=xlsx'
df = pd.read_excel(url)
print(df.head())

            Timestamp      ID CQ_selection  Q1 Q1_free  Q2 Q2_free
0 2023-01-01 00:00:00  Panel1        CQ_A1   6   とても良い   7      普通
1 2023-01-01 01:00:00  Panel1        CQ_A2   6    おいしい   6      普通
2 2023-01-01 02:00:00  Panel1        CQ_A3   6   とても良い   7   つまらない
3 2023-01-01 03:00:00  Panel1        CQ_A4   4     ふつう   7      退屈
4 2023-01-01 04:00:00  Panel1        CQ_A5   7    おいしい   6      最高


読み込んだデータが正しいか確認ください。

## 関数の作成
解析に必要な関数を作成します。理解は不要です。実行だけしてください。

In [5]:
def add_id_check_slide(prs, df_cq, cq):
    """Add a slide to check ID consistency."""
    ids = df_cq['ID']
    total_ids = len(ids)
    unique_ids = ids.nunique()

    if total_ids == 15 and unique_ids == 15:
        result_text = "全員回答済みで重複なし"
    elif total_ids == 15 and unique_ids < 15:
        result_text = "重複あり"
    elif total_ids < 15:
        result_text = "未回答あり"
    elif total_ids > 15:
        result_text = "重複回答あり"

    slide = prs.slides.add_slide(prs.slide_layouts[5])
    tx_box = slide.shapes.add_textbox(Inches(1), Inches(2), Inches(8), Inches(4))
    tf = tx_box.text_frame
    p = tf.add_paragraph()
    p.text = result_text
    p.font.bold = True
    p.font.size = Pt(24)
    title_slide = slide.shapes.title
    title_slide.text = f"{cq} ID Check Results"

def analyze_questions(question_data):
    median = np.median(question_data)
    mean = np.mean(question_data)
    mad = np.mean(np.abs(question_data - median))
    lower = np.percentile(question_data, 30)
    upper = np.percentile(question_data, 70)
    ipr = upper - lower
    ipr_cp = (lower + upper) / 2
    asymmetry_index = abs(5 - ipr_cp)
    ipras = 2.35 + (1.5 * asymmetry_index)
    matching = "不一致" if ipr > ipras else "一致"
    mismatch_index = ipr / ipras
    over_7_count = np.sum(question_data >= 7)
    over_7_percentage = np.mean(question_data >= 7) * 100
    consensus = "合意" if median >= 7 and mismatch_index < 1 else "不合意"

    # 判断
    if median >= 7:
        judgment = '適切'
    elif median >= 4:
        judgment = '不確実'
    else:
        judgment = '不適切'

    return {
        '中央値': median,
        '平均': mean,
        '判断': judgment,
        'Mean Absolute Deviation': mad,
        'Lower': lower,
        'Upper': upper,
        'IPR': ipr,
        'IPR CP': ipr_cp,
        'Asymmetry Index': asymmetry_index,
        'IPRAS': ipras,
        '一致': matching,
        '不一致指数': mismatch_index,
        '7点以上の人数': over_7_count,
        '7点以上の割合': over_7_percentage,
        '合意': consensus
    }


def add_histogram_slide(prs, data, title):
    """Generate and add a histogram slide to the presentation."""
    fig, ax = plt.subplots(figsize=(5, 4))
    ax.hist(data, bins=np.arange(0.5, 10.5, 1), alpha=0.7, color='blue')
    ax.set_title(title + ' Score Distribution')
    ax.set_xlabel('Score')
    ax.set_ylabel('Frequency')

    # Save the plot to a BytesIO object and insert it into the slide
    plot_stream = io.BytesIO()
    plt.savefig(plot_stream, format='png')
    plot_stream.seek(0)
    slide = prs.slides.add_slide(prs.slide_layouts[5])  # Adjust layout if necessary
    slide.shapes.add_picture(plot_stream, Inches(1), Inches(1), width=Inches(8))
    plt.close()

def add_statistical_summary_slide(prs, results, title):
    """Add a slide with statistical summaries to the presentation."""
    slide = prs.slides.add_slide(prs.slide_layouts[5])
    title_slide = slide.shapes.title
    title_slide.text = title
    tx_box = slide.shapes.add_textbox(Inches(1), Inches(1.5), Inches(8), Inches(4))
    tf = tx_box.text_frame

    for key, value in results.items():
        p = tf.add_paragraph()
        # Check if the value is a number (int or float) and format accordingly
        if isinstance(value, (int, float)):
            formatted_value = f"{value:.3g}"  # Format numbers to 3 significant digits
        else:
            formatted_value = value  # Use the string as-is
        p.text = f"{key}: {formatted_value}"
        p.font.size = Pt(20)

def add_text_slide(prs, title, text):
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    title_slide = slide.shapes.title
    title_slide.text = title
    tx_box = slide.shapes.add_textbox(Inches(1), Inches(2), Inches(8), Inches(4))
    tf = tx_box.text_frame
    p = tf.add_paragraph()
    p.text = text

def prepare_presentation(df, cq_selections):
    for cq in cq_selections:
        prs = Presentation()

        df_cq = df[df['CQ_selection'] == cq]

        # タイトルスライドの追加
        slide_layout = prs.slide_layouts[0]  # 0 はタイトルスライド
        slide = prs.slides.add_slide(slide_layout)
        title = slide.shapes.title
        title.text = f"{cq}の解析結果"

        # Task 1: ID Check
        add_id_check_slide(prs, df_cq, cq)

        # Generate histograms for Q1 and Q2
        add_histogram_slide(prs, df_cq['Q1'], f"{cq} Q1")
        add_histogram_slide(prs, df_cq['Q2'], f"{cq} Q2")

        # Statistical summary for Q1 and Q2
        results_q1 = analyze_questions(df_cq['Q1'])
        results_q2 = analyze_questions(df_cq['Q2'])
        add_statistical_summary_slide(prs, results_q1, f"{cq} Q1 Statistical Summary")
        add_statistical_summary_slide(prs, results_q2, f"{cq} Q2 Statistical Summary")

        # Summarize free text for Q1_free and Q2_free
        q1_free_text = "\n".join(df_cq['Q1_free'])
        q2_free_text = "\n".join(df_cq['Q2_free'])
        add_text_slide(prs, f"{cq} Q1 Free Text Responses", q1_free_text)
        add_text_slide(prs, f"{cq} Q2 Free Text Responses", q2_free_text)


        # Save the completed presentation to a file
        base_path = "/content/drive/MyDrive/yourpass"
        filename = f"{base_path}/{cq}_output.pptx"
        prs.save(filename)
        print(f"Saved presentation for {cq} as {filename}")

# Example usage
# Assume 'df' is your DataFrame containing all the necessary data
# cq_selections = ['CQ_A2', 'CQ_A3', ..., 'CQ_A10']
# prepare_presentation(df, cq_selections)



上までを走らせて関数を記憶させたら準備は完了です。

あとは、CQ_A1からCQ_A10まで完了したところを実行していくと、フォルダ内にスライドが格納されます。


# 投票後の実行
以下のコードを投票後に実施していくと、フォルダ内に自動でスライドが追加されます。あとはフォルダからスライドをみればOKです。
データは逐次更新を想定しているため、毎回読み込み直します

## CQ_A1

In [ ]:
# CQ categories you want to analyze
cq_selections = ['CQ_A1']  # Extend this list as needed

# Call the function
prepare_presentation(df, cq_selections)


Saved presentation for CQ_A1 as /content/drive/MyDrive/yourpass/CQ_A1_output.pptx


## CQ_A2

In [6]:
url = 'https://docs.google.com/spreadsheets/d/1MXIEA-pUss9VBQ1nVzmk83AKdXQwwOw1YbWLz_4LTMg/export?format=xlsx'
df = pd.read_excel(url)

# CQ categories you want to analyze
cq_selections = ['CQ_A2']  # Extend this list as needed

# Call the function
prepare_presentation(df, cq_selections)


Saved presentation for CQ_A2 as /content/drive/MyDrive/yourpass/CQ_A2_output.pptx


## CQ_A3

In [7]:
url = 'https://docs.google.com/spreadsheets/d/1MXIEA-pUss9VBQ1nVzmk83AKdXQwwOw1YbWLz_4LTMg/export?format=xlsx'
df = pd.read_excel(url)

# CQ categories you want to analyze
cq_selections = ['CQ_A3']  # Extend this list as needed

# Call the function
prepare_presentation(df, cq_selections)


Saved presentation for CQ_A3 as /content/drive/MyDrive/yourpass/CQ_A3_output.pptx


## CQ_A4

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1MXIEA-pUss9VBQ1nVzmk83AKdXQwwOw1YbWLz_4LTMg/export?format=xlsx'
df = pd.read_excel(url)

# CQ categories you want to analyze
cq_selections = ['CQ_A4']  # Extend this list as needed

# Call the function
prepare_presentation(df, cq_selections)


## CQ_A5

In [8]:
url = 'https://docs.google.com/spreadsheets/d/1MXIEA-pUss9VBQ1nVzmk83AKdXQwwOw1YbWLz_4LTMg/export?format=xlsx'
df = pd.read_excel(url)

# CQ categories you want to analyze
cq_selections = ['CQ_A5']  # Extend this list as needed

# Call the function
prepare_presentation(df, cq_selections)


Saved presentation for CQ_A5 as /content/drive/MyDrive/yourpass/CQ_A5_output.pptx


## CQ_A6

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1MXIEA-pUss9VBQ1nVzmk83AKdXQwwOw1YbWLz_4LTMg/export?format=xlsx'
df = pd.read_excel(url)

# CQ categories you want to analyze
cq_selections = ['CQ_A6']  # Extend this list as needed

# Call the function
prepare_presentation(df, cq_selections)


## CQ_A7

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1MXIEA-pUss9VBQ1nVzmk83AKdXQwwOw1YbWLz_4LTMg/export?format=xlsx'
df = pd.read_excel(url)

# CQ categories you want to analyze
cq_selections = ['CQ_A7']  # Extend this list as needed

# Call the function
prepare_presentation(df, cq_selections)


## CQ_A8

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1MXIEA-pUss9VBQ1nVzmk83AKdXQwwOw1YbWLz_4LTMg/export?format=xlsx'
df = pd.read_excel(url)

# CQ categories you want to analyze
cq_selections = ['CQ_A8']  # Extend this list as needed

# Call the function
prepare_presentation(df, cq_selections)


## CQ_A9

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1MXIEA-pUss9VBQ1nVzmk83AKdXQwwOw1YbWLz_4LTMg/export?format=xlsx'
df = pd.read_excel(url)

# CQ categories you want to analyze
cq_selections = ['CQ_A9']  # Extend this list as needed

# Call the function
prepare_presentation(df, cq_selections)


## CQ_A10

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1MXIEA-pUss9VBQ1nVzmk83AKdXQwwOw1YbWLz_4LTMg/export?format=xlsx'
df = pd.read_excel(url)

# CQ categories you want to analyze
cq_selections = ['CQ_A10']  # Extend this list as needed

# Call the function
prepare_presentation(df, cq_selections)
